https://drive.google.com/file/d/1Ug-NMWpOrPiOr9TDVuJ103bfeDl8VeNR/view?usp=sharing

In [94]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]


In [95]:
Y_r

,level_0,level_1,0,Price
0,0,EURPrices+0,24.65,24.65
1,0,EURPrices+1,21.59,21.59
2,0,EURPrices+2,18.29,18.29
3,0,EURPrices+3,14.12,14.12
4,0,EURPrices+4,11.58,11.58
...,...,...,...,...
8755,364,EURPrices+19,105.00,105.00
8756,364,EURPrices+20,94.50,94.50
8757,364,EURPrices+21,84.90,84.90
8758,364,EURPrices+22,87.55,87.55


In [96]:
def run_electricity_strategy(df, eff, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_df.loc[min_price_index, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            charge_level=charge_level
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01['Price'].idxmax()
                min_price_index_4 = remaining_prices_01['Price'].idxmin()
                if current_df.loc[min_price_index_4, 'Price'] < current_df.loc[max_price_index_4, 'Price'] * eff:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                    
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1['Price'].idxmax()
            min_price_index_2 = remaining_prices_1['Price'].idxmin()
            if current_df.loc[min_price_index_2, 'Price'] < current_df.loc[max_price_index_2, 'Price'] * eff:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                    
        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2['Price'].idxmax()
            min_price_index_3 = remaining_prices_2['Price'].idxmin()
            if current_df.loc[min_price_index_3, 'Price'] < current_df.loc[max_price_index_3, 'Price'] * eff:
                  if min_price_index_3 < max_price_index_3:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  elif min_price_index_3 > max_price_index_3:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  else:
                        continue         

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, eff=1, capacity=10,charge_level=4, ramp_rate=4, min_charge_level=1)


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,4,11.58,16,31.66,80.32,4
1,5,12.26,9,25.83,54.28,4
2,6,15.34,9,23.22,31.52,4
3,23,13.98,17,31.66,39.06,5
4,3,14.12,0,24.65,42.12,5
...,...,...,...,...,...,...
1647,8713,81.21,8712,81.90,2.76,5
1648,8739,81.06,8745,148.90,271.36,5
1649,8742,81.53,8744,121.10,158.28,5
1650,8759,83.68,8746,135.88,208.80,5


In [97]:
print(sum(run_electricity_strategy(df=Y_r, eff=1, capacity=10,charge_level=4, ramp_rate=4, min_charge_level=1).iloc[:,4:5].values))

[297255.5]


In [ ]:
def run_electricity_strategy(df, eff, capacity, ramp_rate, min_charge_level):
    prices = []
    charge_level = 1    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_df.loc[min_price_index, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
       

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, eff=1, capacity=5, ramp_rate=2, min_charge_level=1)

In [ ]:
def run_electricity_strategy(df, eff, capacity, ramp_rate, min_charge_level):
    prices = []
    charge_level = 5
    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        if min_price_index < max_price_index:
            bottleneck_1 = min(capacity - charge_level, ramp_rate)
            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
            charge_level += bottleneck_1
            charge_level -= bottleneck_2
            
            profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
            prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], 
                           max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
        elif min_price_index > max_price_index:
            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
            charge_level -= bottleneck_2
            bottleneck_1 = min(capacity - charge_level, ramp_rate)
            charge_level += bottleneck_1
            
            profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
            prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], 
                           max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
        else:
            continue

    return prices
run_electricity_strategy(df=Y_r, eff=1, capacity=5, ramp_rate=2, min_charge_level=1)

In [54]:
def run_electricity_strategy(df, eff, capacity, ramp_rate, min_charge_level):
    prices = []
    charge_level = 5

    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]

        if len(remaining_prices_0) > 1:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                max_price = current_df.loc[max_price_index, 'Price']
                min_price = current_df.loc[min_price_index, 'Price']
                profit = (max_price * bottleneck_2 * eff) - (min_price * bottleneck_1)
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, max_price, bottleneck_1, bottleneck_2, profit, charge_level))
                
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                max_price_1 = remaining_prices_0['Price'].max()
                min_price_1 = remaining_prices_0['Price'].min()
                profit_1 = (max_price_1 * bottleneck_2 * eff) - (min_price_1 * bottleneck_1)
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], remaining_prices_0['Price'].idxmax(), max_price_1, bottleneck_1, bottleneck_2, profit_1, charge_level))
                
            else:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                max_price_1 = remaining_prices_0['Price'].max()
                min_price_1 = remaining_prices_0['Price'].min()
                profit_1 = (max_price_1 * bottleneck_2 * eff) - (min_price_1 * bottleneck_1)
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], bottleneck_1, bottleneck_2, profit_1, charge_level))
                
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                max_price = current_df.loc[max_price_index, 'Price']
                min_price = current_df.loc[min_price_index, 'Price']
                profit = (max_price * bottleneck_2 * eff) - (min_price * bottleneck_1)
                prices.append((remaining_prices_0['Price'].idxmin(), remaining_prices_0['Price'].min(), max_price_index, current_df.loc[max_price


run_electricity_strategy(df=Y_r, eff=1, capacity=5, ramp_rate=2, min_charge_level=1)

SyntaxError: invalid syntax (3525524866.py, line 56)

In [ ]:

profitSpecific(maxHourIndex)=(price(maxHourIndex)*eff-price(minHourIndex)); 
profitSpecific(minHourIndex)=-profitSpecific(maxHourIndex)

if price(maxHourIndex)*eff> price(minHourIndex)
    if minHourIndex<maxHourIndex 
        storageFree=capacity-max(chargeLevel(minHourIndex:maxHourIndex));
        bottleneck=min([TAvail(maxHourIndex),PAvail(minHourIndex),storageFree/periodT]); 
            
    else if minHourIndex>maxHourIndex 
        storageLeft=min(chargeLevel(maxHourIndex:minHourIndex));
        bottleneck=min([TAvail(maxHourIndex),PAvail(minHourIndex),storageLeft/periodT]); 
    end
        
        
TAvail(maxHourIndex)=TAvail(maxHourIndex)-bottleneck;   
PAvail(minHourIndex)=PAvail(minHourIndex)-bottleneck;   
Tuse(maxHourIndex)=Tuse(maxHourIndex)+bottleneck;       
Puse(minHourIndex)=Puse(minHourIndex)+bottleneck;      
        
chargeLevel(maxHourIndex+1:end)=chargeLevel(maxHourIndex+1:end)-bottleneck*periodT;
chargeLevel(minHourIndex+1:end)=chargeLevel(minHourIndex+1:end)+bottleneck*periodT;

profit(maxHourIndex)=Tuse(maxHourIndex)*periodT*profitSpecific(maxHourIndex);  
profit(minHourIndex)=-profit(maxHourIndex);

profitTotal=profitTotal+profit(maxHourIndex);


In [62]:
def run_electricity_strategy(df, eff, capacity, ramp_rate, min_charge_level):
    prices = []
    charge_level = 5    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_df.loc[min_price_index, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
            
            
            
            
           
            
            
            
            
            
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    
                    
                    
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01['Price'].idxmax()
                min_price_index_4 = remaining_prices_01['Price'].idxmin()
                if current_df.loc[min_price_index_4, 'Price'] < current_df.loc[max_price_index_4, 'Price'] * eff:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                    
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1['Price'].idxmax()
            min_price_index_2 = remaining_prices_1['Price'].idxmin()
            if current_df.loc[min_price_index_2, 'Price'] < current_df.loc[max_price_index_2, 'Price'] * eff:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                    
        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2['Price'].idxmax()
            min_price_index_3 = remaining_prices_2['Price'].idxmin()
            if current_df.loc[min_price_index_3, 'Price'] < current_df.loc[max_price_index_3, 'Price'] * eff:
                  if min_price_index_3 < max_price_index_3:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  elif min_price_index_3 > max_price_index_3:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  else:
                        continue         
                        
    
    

    return prices
run_electricity_strategy(df=Y_r, eff=1, capacity=5, ramp_rate=2, min_charge_level=1)

[(4, 11.58, 16, 31.66, 63.32, 3),
 (5, 12.26, 9, 25.83, 27.139999999999997, 3),
 (6, 15.34, 9, 23.22, 15.759999999999998, 3),
 (23, 13.98, 17, 31.66, 35.36, 3),
 (3, 14.12, 0, 24.65, 21.06, 3)]

In [59]:
def run_electricity_strategy(df, eff):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], 
                               max_price_index, current_df.loc[max_price_index, 'Price']))

                prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], 
                               max_price_index_1, current_df.loc[max_price_index_1, 'Price']))

                smaller_index_1 = min(min_price_index_1, max_price_index_1)
                larger_index_1 = max(min_price_index_1, max_price_index_1)
                remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
                if len(remaining_prices_01) > 1:
                    max_price_index_4 = remaining_prices_01['Price'].idxmax()
                    min_price_index_4 = remaining_prices_01['Price'].idxmin()
                    if current_df.loc[min_price_index_4, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], 
                                       max_price_index_4, current_df.loc[max_price_index_4, 'Price']))           
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1['Price'].idxmax()
            min_price_index_2 = remaining_prices_1['Price'].idxmin()
            if current_df.loc[min_price_index_2, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], 
                               max_price_index_2, current_df.loc[max_price_index_2, 'Price']))

        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2['Price'].idxmax()
            min_price_index_3 = remaining_prices_2['Price'].idxmin()
            if current_df.loc[min_price_index_3, 'Price'] < current_df.loc[max_price_index_3, 'Price'] * eff:
                prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], 
                               max_price_index_3, current_df.loc[max_price_index_3, 'Price']))

                               
    return prices

run_electricity_strategy(df=Y_r, eff=1)

TypeError: run_electricity_strategy() missing 3 required positional arguments: 'capacity', 'ramp_rate', and 'min_charge_level'